In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import sys
sys.path.append('..')
from dataset import batcher

In [4]:
MODEL_DIR = '../saved_models/grid_search/'
LOSS = 'MeanSquaredError'
METRICS = ['RootMeanSquaredError']
COUNTRIES = ['ethiopia-2018', 'nigeria-2018', 'malawi-2016']

In [5]:
# Get test split
# bands = ['BLUE', 'GREEN', 'RED', 'NIR', 'SW_IR1', 'SW_IR2', 'TEMP', 'VIIRS', 'DELTA_TEMP', 'CO']
def batch(country_year):
    bands = ['VIIRS']
    test_batcher = batcher.Batcher(bands, country_year=country_year, bucket=False, shuffle=False, split='custom', repeat=1, batch_size=3000).get_dataset()
    x_true = []
    y_true = []
    for x, y in test_batcher:
        x_true.append(x)
        y_true.append(y)

    y_true = np.squeeze(y_true)
    return x_true, y_true

In [6]:
# Load model
def load_model_from_dir(model_name):
    model_path = MODEL_DIR + model_name + '.h5'
    model = tf.keras.models.load_model(model_path, compile=False)
    
    return model

In [7]:
# Testing purposes only
# grid_search_results = pd.DataFrame(columns=['experiment', 'epoch', 'learning_rate', 'r2'])
# results = {'experiment': "imagery_sample_cnn_regression_0.01_40", 'epoch': 50, 'learning_rate': 1e-2, 'r2': 0.23}
# grid_search_results = grid_search_results.append(results, ignore_index = True)
# display(grid_search_results)

In [20]:
# Hyperparamters tested
lrs = [1e-1, 1e-2, 1e-3, 1e-4]
epochs = [10, 50, 100, 200]

def run_grid_search(country_year):
    grid_search_results = pd.DataFrame(columns=['experiment', 'epoch', 'learning_rate', 'r2'])
    x_true, y_true = batch(country_year)

    for e in epochs:
        for l in lrs:
            experiment_name='imagery_sample_cnn_regression' + str(e) + '_' + str(l)

            # Set hyperparamters
            lr_rate = l
            opt = tf.keras.optimizers.Adam(learning_rate=lr_rate)

            # Get consumption predictions
            trained_model = load_model_from_dir(experiment_name)
            trained_model.compile(
                        loss=LOSS,
                        optimizer=opt,
                        metrics=METRICS,
            )

            y_preds = trained_model.predict_on_batch(x_true)
            y_preds = np.squeeze(y_preds)
            r2 = pearsonr(y_true, y_preds)[0]

            results = {'experiment': experiment_name, 'epoch': e, 'learning_rate': l, 'r2': r2}
            grid_search_results = grid_search_results.append(results, ignore_index = True)
            
    return grid_search_results


In [21]:
for country_year in COUNTRIES:  
    country_grid_search_results = run_grid_search(country_year)
    print(f'{country_year}:')
    print(country_grid_search_results.sort_values(by=['r2']))

2023-02-22 17:12:31.523927: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/var/folders/yw/3z56n_vx4vq9g05qrxzrr4m40000gn/T/ipykernel_7792/3769678599.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grid_search_results = grid_search_results.append(results, ignore_index = True)
2023-02-22 17:12:34.772133: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/var/folders/yw/3z56n_vx4vq9g05qrxzrr4m40000gn/T/ipykernel_7792/3769678599.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grid_search_results = grid_search_results.append(results, ignore_index = True)
2023-02-22 17:12:37.462892: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.

ethiopia-2018:
                                 experiment epoch  learning_rate        r2
3    imagery_sample_cnn_regression10_0.0001    10         0.0001 -0.346282
10   imagery_sample_cnn_regression100_0.001   100         0.0010 -0.334725
6     imagery_sample_cnn_regression50_0.001    50         0.0010 -0.308964
11  imagery_sample_cnn_regression100_0.0001   100         0.0001 -0.257944
9     imagery_sample_cnn_regression100_0.01   100         0.0100 -0.211487
4       imagery_sample_cnn_regression50_0.1    50         0.1000 -0.137019
12     imagery_sample_cnn_regression200_0.1   200         0.1000 -0.133705
1      imagery_sample_cnn_regression10_0.01    10         0.0100 -0.101564
0       imagery_sample_cnn_regression10_0.1    10         0.1000 -0.079085
5      imagery_sample_cnn_regression50_0.01    50         0.0100 -0.040384
13    imagery_sample_cnn_regression200_0.01   200         0.0100  0.016313
14   imagery_sample_cnn_regression200_0.001   200         0.0010  0.111184
2     imag

2023-02-22 17:13:15.395058: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/var/folders/yw/3z56n_vx4vq9g05qrxzrr4m40000gn/T/ipykernel_7792/3769678599.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grid_search_results = grid_search_results.append(results, ignore_index = True)
2023-02-22 17:13:17.925116: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/var/folders/yw/3z56n_vx4vq9g05qrxzrr4m40000gn/T/ipykernel_7792/3769678599.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grid_search_results = grid_search_results.append(results, ignore_index = True)
2023-02-22 17:13:20.444646: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.

nigeria-2018:
                                 experiment epoch  learning_rate        r2
13    imagery_sample_cnn_regression200_0.01   200         0.0100 -0.328629
6     imagery_sample_cnn_regression50_0.001    50         0.0010 -0.281024
3    imagery_sample_cnn_regression10_0.0001    10         0.0001 -0.254306
14   imagery_sample_cnn_regression200_0.001   200         0.0010 -0.214982
11  imagery_sample_cnn_regression100_0.0001   100         0.0001 -0.214489
2     imagery_sample_cnn_regression10_0.001    10         0.0010 -0.108389
12     imagery_sample_cnn_regression200_0.1   200         0.1000 -0.102902
10   imagery_sample_cnn_regression100_0.001   100         0.0010 -0.007413
1      imagery_sample_cnn_regression10_0.01    10         0.0100  0.046697
4       imagery_sample_cnn_regression50_0.1    50         0.1000  0.048577
8      imagery_sample_cnn_regression100_0.1   100         0.1000  0.048891
5      imagery_sample_cnn_regression50_0.01    50         0.0100  0.144960
0       ima

2023-02-22 17:13:59.903042: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/var/folders/yw/3z56n_vx4vq9g05qrxzrr4m40000gn/T/ipykernel_7792/3769678599.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grid_search_results = grid_search_results.append(results, ignore_index = True)
2023-02-22 17:14:02.462824: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/var/folders/yw/3z56n_vx4vq9g05qrxzrr4m40000gn/T/ipykernel_7792/3769678599.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grid_search_results = grid_search_results.append(results, ignore_index = True)
2023-02-22 17:14:04.986314: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.

malawi-2016:
                                 experiment epoch  learning_rate        r2
3    imagery_sample_cnn_regression10_0.0001    10         0.0001 -0.429320
11  imagery_sample_cnn_regression100_0.0001   100         0.0001 -0.406226
6     imagery_sample_cnn_regression50_0.001    50         0.0010 -0.325102
10   imagery_sample_cnn_regression100_0.001   100         0.0010 -0.304028
4       imagery_sample_cnn_regression50_0.1    50         0.1000 -0.241098
0       imagery_sample_cnn_regression10_0.1    10         0.1000 -0.202642
5      imagery_sample_cnn_regression50_0.01    50         0.0100 -0.149080
2     imagery_sample_cnn_regression10_0.001    10         0.0010 -0.126365
9     imagery_sample_cnn_regression100_0.01   100         0.0100 -0.122072
1      imagery_sample_cnn_regression10_0.01    10         0.0100 -0.029470
8      imagery_sample_cnn_regression100_0.1   100         0.1000 -0.004301
12     imagery_sample_cnn_regression200_0.1   200         0.1000  0.091406
13    imager

/var/folders/yw/3z56n_vx4vq9g05qrxzrr4m40000gn/T/ipykernel_7792/3769678599.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grid_search_results = grid_search_results.append(results, ignore_index = True)


In [ ]:
"""
Results:

"""